# PGURE-SVT Demonstration

PGURE-SVT (Poisson-Gaussian Unbiased Risk Estimator - Singular Value Thresholding) is an algorithm designed to denoise image sequences acquired in microscopy. It exploits the correlations between consecutive frames to form low-rank matrices, which are then recovered using a technique known as nuclear norm minimization. An unbiased risk estimator for mixed Poisson-Gaussian noise is used to automate the selection of the regularization parameter, while robust noise and motion estimation maintain broad applicability to many different types of microscopy.

You can read more about the algorithm and its applications in:

> T. Furnival, R. K. Leary and P. A. Midgley, "Denoising time-resolved microscopy sequences with singular value thresholding", *Ultramicroscopy*, vol. 178, pp. 112–124, 2017. DOI:[10.1016/j.ultramic.2016.05.005](http://dx.doi.org/10.1016/j.ultramic.2016.05.005)

This example notebook shows how PGURE-SVT can be combined with [HyperSpy](http://hyperspy.org), which is an open-source Python library that makes signal handling and processing straightforward in Python.

In [ ]:
%matplotlib notebook

import numpy as np
import hyperspy.api as hs

from pguresvt import hspysvt, mixed_noise_model

## 1. Simulated dataset

First, we load the simulated dataset using HyperSpy.

In [ ]:
# Load example dataset
s = hs.load("example.tif")

# Truncate to 25 frames
s = s.inav[:25]

# Plot the result
s.plot(navigator='slider')

Now we corrupt the movie with using a noise generator for mixed Poisson-Gaussian noise, according to the following equation, where the true, noise-free signal is $\mathbf{X}^{0}$, and the observed noisy signal is $\mathbf{Y}$:

$$
\mathbf{Y}=\alpha\mathbf{Z}+\mathbf{E}\;\textrm{ with }\;\begin{cases}
  \mathbf{Z}\thicksim\mathcal{P}\left(\frac{\mathbf{X}^{0}}{\alpha}\right)\\
  \mathbf{E}\thicksim\mathcal{N}\left(\mu,\sigma^{2}\right)
  \end{cases}
$$

where $\alpha$ is the detector gain, $\mu$ is the detector offset, and $\sigma$ is the detector noise. 



In [ ]:
random_seed = 123
detector_gain = 0.1
detector_offset = 0.1
detector_sigma = 0.1

noisy_data = mixed_noise_model(
    s.data,
    alpha=detector_gain,
    mu=detector_offset,
    sigma=detector_sigma,
    random_seed=123,
)

s_noisy = hs.signals.Signal2D(noisy_data)
s_noisy.plot(navigator="slider")

Next we initialise the SVT denoising function. You can evaluate the following cell to view the full docstring.

In [ ]:
??hspysvt.HSPYSVT

In this example we do not use the noise estimation procedure, and instead provide the known parameters to the algorithm directly. This information is used by the PGURE optimizer to calculate the threshold.

In [ ]:
# Initialize with suggested parameters
svt = hspysvt.HSPYSVT(
    patch_size=4,
    noise_alpha=detector_gain,
    noise_mu=detector_offset,
    noise_sigma=detector_sigma,
    tol=1e-5,
)

Now we are able to run the denoising and plot the result:

In [ ]:
# Run the denoising
s_denoised = svt.denoise(s_noisy)

# Plot denoised data
s_denoised.plot(navigator='slider')

## 2. Time-resolved ADF-STEM image sequence

In this example we apply PGURE-SVT to an experimental dataset of a nanoparticle acquired using ADF-STEM. This image sequence contains 51 frames at a rate of 4 frames per second. The results of this denoising are shown in Fig. 11 of [the paper](http://dx.doi.org/10.1016/j.ultramic.2016.05.005).

For larger images, such as the 256x256 pixels here, you can use the `patch_overlap` parameter to control the trade-off between speed and accuracy of the denoising procedure. This reduces the number of patches the algorithm works with, at the expense of introducing possible edge artefacts between patches.

For the experimental sequence, the detector offset (`noise_mu`) was known beforehand, so a noise estimation procedure is used for the other values.

In [ ]:
# Load example dataset and plot
s_np = hs.load("nanoparticle.tif")
s_np.plot(navigator="slider")

In [ ]:
# Initialize with suggested parameters, optimized for speed
expt_svt = hspysvt.HSPYSVT(patch_size=4, patch_overlap=2, noise_mu=0.075)

# Run the denoising
s_np_denoised = expt_svt.denoise(s_np)

# Plot denoised data
s_np_denoised.plot(navigator="slider")

The parameters used to generate Fig. 11 in the paper are given below. Note that using these values can be *slow*, taking ~30 seconds per frame.
```
expt_svt = hspysvt.HSPYSVT(patchsize=4,
                           patchoverlap=1,
                           mu=0.075,
                           tol=1e-8,
                           arpssize=11,
                           numthreads=4)
```